In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


data = load_iris()
X, y = data.data, data.target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from scipy.stats import uniform, randint

search_space = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 10),
    'max_features': uniform(0.1, 0.9)  
}


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_auc_score
import numpy as np

def clamp(value, min_value, max_value):
    return max(min_value, min(value, max_value))

def objective_function(params, X, y):
    n_estimators = int(clamp(params['n_estimators'], 50, 200))
    max_depth = int(clamp(params['max_depth'], 3, 15))
    min_samples_split = int(clamp(params['min_samples_split'], 2, 10))
    max_features = clamp(params['max_features'], 0.1, 1.0)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=42
    )
    lb = LabelBinarizer()
    y_bin = lb.fit_transform(y)
    cv_scores = []

    skf = StratifiedKFold(n_splits=5)
    for train_idx, test_idx in skf.split(X, y):
        model.fit(X[train_idx], y[train_idx])
        y_prob = model.predict_proba(X[test_idx])
        
        if y_bin.shape[1] == 1:
            roc_auc = roc_auc_score(y[test_idx], y_prob[:, 1])
        else:
            roc_auc = roc_auc_score(y_bin[test_idx], y_prob, average='macro')
        
        cv_scores.append(roc_auc)

    return -np.mean(cv_scores)


In [ ]:
import random

def get_random_hyperparameters(search_space):
    return {key: dist.rvs() for key, dist in search_space.items()}

def initial_observations(n, search_space, X, y):
    observations = []
    for _ in range(n):
        params = get_random_hyperparameters(search_space)
        score = objective_function(params, X, y)
        observations.append((params, score))
    return observations


In [ ]:
from sklearn.neighbors import KernelDensity

def fit_kde(observations):
    samples = np.array([list(obs[0].values()) for obs in observations])
    kde = KernelDensity(kernel='gaussian').fit(samples)
    return kde

def sample_from_kde(kde, search_space):
    samples = kde.sample()
    return {key: samples[0][i] for i, key in enumerate(search_space.keys())}


In [ ]:
def optimize_hyperparameters(X, y, search_space, n_initial=10, n_iterations=50):
    observations = initial_observations(n_initial, search_space, X, y)
    
    for _ in range(n_iterations - n_initial):
        sorted_observations = sorted(observations, key=lambda x: x[1])
        split_point = int(len(sorted_observations) * 0.2)
        x1 = sorted_observations[:split_point]
        x2 = sorted_observations[split_point:]
        
        kde_x1 = fit_kde(x1)
        kde_x2 = fit_kde(x2)
        
        params = sample_from_kde(kde_x1, search_space)
        score = objective_function(params, X, y)
        observations.append((params, score))
    
    best_params = sorted(observations, key=lambda x: x[1])[0][0]
    return best_params


best_hyperparameters = optimize_hyperparameters(X_train, y_train, search_space)
print("Best Hyperparameters: ", best_hyperparameters)


In [ ]:
best_model = RandomForestClassifier(
    n_estimators=int(clamp(best_hyperparameters['n_estimators'], 50, 200)),
    max_depth=int(clamp(best_hyperparameters['max_depth'], 3, 15)),
    min_samples_split=int(clamp(best_hyperparameters['min_samples_split'], 2, 10)),
    max_features=clamp(best_hyperparameters['max_features'], 0.1, 1.0),
    random_state=42
)

best_model.fit(X_train, y_train)
y_pred_proba = best_model.predict_proba(X_test)

roc_auc = roc_auc_score(LabelBinarizer().fit_transform(y_test), y_pred_proba, average='macro')
print(f"ROC AUC on Test Set: {roc_auc:.4f}")
